# Symbolic Partial Derivative Routine

## Authors: Zach Etienne & Tyler Knowles

## This module contains a routine for computing an analytic partial derivative of a mathematical expression that is written as seveal subexpressions.

**Notebook Status:** <font color='red'><b> In progress </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). Additionally, this notebook has been validated by checking that results are consistent with finite-difference derivative values in [LALSuite](https://git.ligo.org/lscsoft/lalsuite).

### NRPy+ Source Code for this module: [SEOBNR_Derivative_Routine.py](../../edit/in_progress/SEOBNR/SEOBNR_Derivative_Routine.py)

## Introduction
$$\label{intro}$$

This notebook documents the symbolic partial derivative routine used to generate analytic derivatives of the [SEOBNRv3](https://git.ligo.org/lscsoft/lalsuite) Hamiltonian (documented [here](../Tutorial-SEOBNR_Documentation.ipynb)) and described in [this article](https://arxiv.org/abs/1803.06346).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Initialize core Python/NRPy+ modules
1. [Step 2:](#step2) Read in expressions
1. [Step 3:](#step3) Read in constants
1. [Step 4:](#step4) List free symbols
1. [Step 5:](#step5) Convert expressions to function notation
1. [Step 6:](#step6) Differentiate with respect to xx
1. [Step 7:](#step7) Simplify derivative expressions
1. [Step 9:](#step9) Differentiate with respect to a specific free variable
1. [Step 10:](#step10) Compute derivatives with respect to each free variable
1. [Step 11:](#step11) Output result
1. [Step 12:](#code_validation): Code Validation against `SEOBNR_Derivative_Routine` NRPy+ module
1. [Step 13:](#latex_pdf_output) Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from Python/NRPy+:

In [1]:
# Step 1.a: import all needed modules from Python/NRPy+:
import sympy as sp                # SymPy: The Python computer algebra package upon which NRPy+ depends
import os, sys                    # Standard Python modules for multiplatform OS-level functions

# Step 1.?: check system path so can use outputC; #TylerK: remove and put outputC back with other imports
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import *             # TylerK: check what is imported and remove *; also find appropriate description

<a id='step2'></a>

# Step 2: Read in expressions \[Back to [top](#toc)\]
$$\label{step2}$$

We read in the expressions of which we will compute partial derivatives in a single large string before splitting the string by line (carriage return) and by "=".  Doing so allows us to manipulate the right- and left-hand sides of the expressions appropriately.  We store the left- and right-hand sides in the array $\texttt{lr}$, which consists of $\texttt{lhrh}$ arrays with left-hand sides $\texttt{lhs}$ and right-hand sides $\texttt{rhs}$.  Note that $\texttt{Lambda}$ is a protected keyword in Python, so the variable $\Lambda$ in the Hamiltonian is renamed $\texttt{Lamb}$.

In [2]:
# Step 2.a: Read in expressions as a (single) string
with open('SEOBNR/Hamstring_lite.txt', 'r') as file:
    all_expressions = file.read()

# Step 2.b: Split the expression string by carriage returns
string_lines = all_expressions.splitlines()

# Step 2.c: Create and populate the "lr" array, which separates each line into left- and right-hand sides
#   Each entry is a string of the form lhrh(lhs='',rhs='')
lr = []

for i in range(len(string_lines)):
    # Ignore lines with 2 or fewer characters and those starting with #
    if len(string_lines[i]) > 2 and string_lines[i][0] != "#":
        # Split each line by its equals sign
        split_line = string_lines[i].split("=")
        # Append the line to "lr", removing spaces, "sp." prefixes, and replacing Lambda->Lamb
        #   (Lambda is a protected keyword):
        lr.append(lhrh(lhs=split_line[0].replace(" ","").replace("Lambda","Lamb"),
                       rhs=split_line[1].replace(" ","").replace("sp.","").replace("Lambda","Lamb")))

# Step 2.d: Separate and simplify right- and left-hand sides into separate arrays
lhss = []
rhss = []
for i in range(len(lr)):
    lhss.append(sp.sympify(lr[i].lhs))
    rhss.append(sp.sympify(lr[i].rhs))

# Step 2.e: Read in variables with which to take derivatives
with open('SEOBNR/Hamstring_variables.txt', 'r') as file:
    variables = file.read()
# Step 2.f: Split the variable string by carriage returns
dynamic_variables = variables.splitlines()

#TylerK: print for debuggin
print("lr = ",lr)
print("string_lines = ",string_lines)
print("lhss = ",lhss)
print("rhss = ",rhss)
print("dynamic_variables = ",dynamic_variables)

lr =  [lhrh(lhs='r', rhs='x*x+y*y+eta'), lhrh(lhs='csi', rhs='px+py*pz'), lhrh(lhs='s1dots2', rhs='s1x*s2x+s1y*s2y+s1z*s2z'), lhrh(lhs='u', rhs='1/r'), lhrh(lhs='omega', rhs='sqrt(r)')]
string_lines =  ['r = x*x + y*y + eta', 'csi = px + py*pz', 's1dots2 = s1x*s2x + s1y*s2y +s1z*s2z', 'u = 1/r', 'omega = sp.sqrt(r)']
lhss =  [r, csi, s1dots2, u, omega]
rhss =  [eta + x**2 + y**2, px + py*pz, s1x*s2x + s1y*s2y + s1z*s2z, 1/r, sqrt(r)]
dynamic_variables =  ['x', 'y', 'z', 'px', 'py', 'pz', 's1x', 's1y', 's1z', 's2x', 's2y', 's2z']


<a id='step3'></a>

# Step 3: Read in constants \[Back to [top](#toc)\]
$$\label{step3}$$

We declare the constant values; derivatives with respect to these variables will be set to zero.

In [3]:
# Step 3.a: Read in constants as a (single) string
with open('SEOBNR/Hamstring_constants.txt', 'r') as file:
    constants = file.read()

# Step 3.b: Split the input string by carriage returns
constants_as_strings = constants.splitlines()

# Step 3.c: Create "input_constants" array and populate with SymPy constants
input_constants = []
for constant in constants_as_strings:
    constant = sp.symbols(constant,real=True)
    input_constants.append(constant)

#TylerK: print for debuggin
print("input_constants = ",input_constants)

input_constants =  [eta]


<a id='step4'></a>

# Step 4: List free symbols \[Back to [top](#toc)\]
$$\label{step4}$$

By ''free symbols'' we mean the variables in the right-hand sides.  We first create a list of all such terms (using SymPy's built-in free_symbol attribute), including duplicates, and then strip the duplicates.  We then remove input constants from the symbol list.

In [4]:
# Step 4.a: Prepare array of "free symbols" in the right-hand side expressions
full_symbol_list_with_dups = []
for i in range(len(lr)):
    for variable in rhss[i].free_symbols:
        full_symbol_list_with_dups.append(variable)

# Step 4.b: Remove duplicate free symbols
full_symbol_list = superfast_uniq(full_symbol_list_with_dups)

# Step 4.c: Remove input constants from symbol list
for inputconst in input_constants:
    for symbol in full_symbol_list:
        if str(symbol) == str(inputconst):
            full_symbol_list.remove(symbol)

# TylerK: print for debuggin
print("full_symbol_list_with_dups = ",full_symbol_list_with_dups)
print("full_symbol_list = ",full_symbol_list)

full_symbol_list_with_dups =  [eta, x, y, pz, py, px, s1y, s2y, s1x, s1z, s2z, s2x, r, r]
full_symbol_list =  [x, y, pz, py, px, s1y, s2y, s1x, s1z, s2z, s2x, r]


<a id='step5'></a>

# Step 5: Convert expressions to function notation \[Back to [top](#toc)\]
$$\label{step5}$$

In order to compute the partial derivative of each right-hand side, we mark each variable (left-hand side) and each free symbol (in right-hand sides) as a function with argument $\texttt{xx}$.

In [5]:
# Step 5.a: Convert each left-hand side to function notation
#   while separating and simplifying left- and right-hand sides
xx = sp.Symbol('xx')
func = []
for i in range(len(lr)):
    func.append(sp.sympify(sp.Function(lr[i].lhs)(xx)))

# TylerK: print for debuggin
print("func = ",func)

# Step 5.b: Mark each free variable as a function with argument xx
full_function_list = []
for symb in full_symbol_list:
    func = sp.sympify(sp.Function(str(symb))(xx))
    full_function_list.append(func)
    for i in range(len(rhss)):
        for var in rhss[i].free_symbols:
            if str(var) == str(symb):
                rhss[i] = rhss[i].subs(var,func)

# TylerK: print for debuggin
print("full_function_list = ",full_function_list)

func =  [r(xx), csi(xx), s1dots2(xx), u(xx), omega(xx)]
full_function_list =  [x(xx), y(xx), pz(xx), py(xx), px(xx), s1y(xx), s2y(xx), s1x(xx), s1z(xx), s2z(xx), s2x(xx), r(xx)]


<a id='step6'></a>

# Step 6: Differentiate with respect to xx \[Back to [top](#toc)\]
$$\label{step6}$$

Now we differentiate the right-hand expressions with respect to $\textrm{xx}$.  We use the SymPy $\texttt{diff}$ command, differentiating with respect to $\texttt{xx}$.  After so doing, we remove $\texttt{(xx)}$ and "Derivative" (which is output by $\texttt{diff}$, and use "prm" suffix to denote the derivative with respect to $\texttt{xx}$.

In [6]:
# Step 6.a: Use SymPy's diff function to differentiate right-hand sides with respect to xx
#   and append "prm" notation to left-hand sides
lhss_deriv = []
rhss_deriv = []
for i in range(len(rhss)):
    lhss_deriv.append(sp.sympify(str(lhss[i])+"prm"))
    newrhs = sp.sympify(str(sp.diff(rhss[i],xx)).replace("(xx)","").replace(", xx","prm").replace("Derivative",""))
    rhss_deriv.append(newrhs)

#TylerK: for debuggin
print("lhss_deriv = ",lhss_deriv)
print("rhss_deriv = ",rhss_deriv)

lhss_deriv =  [rprm, csiprm, s1dots2prm, uprm, omegaprm]
rhss_deriv =  [2*x*xprm + 2*y*yprm, pxprm + py*pzprm + pyprm*pz, s1x*s2xprm + s1xprm*s2x + s1y*s2yprm + s1yprm*s2y + s1z*s2zprm + s1zprm*s2z, -rprm/r**2, rprm/(2*sqrt(r))]


<a id='step7'></a>

# Step 7: Simplify derivative expressions \[Back to [top](#toc)\]
$$\label{step7}$$

We declare a function to simply the derivative expressions.  In particular, we want to remove terms equal to zero.

In [7]:
# Derivative simplification function
def simplify_deriv(lhss_deriv,rhss_deriv):
    # Copy expressions into another array
    lhss_deriv_simp = []
    rhss_deriv_simp = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_simp.append(lhss_deriv[i])
        rhss_deriv_simp.append(rhss_deriv[i])
    # If a right-hand side is 0, substitute value 0 for the corresponding left-hand side in later terms
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == 0:
            for j in range(i+1,len(rhss_deriv_simp)):
                for var in rhss_deriv_simp[j].free_symbols:
                    if str(var) == str(lhss_deriv_simp[i]):
                        rhss_deriv_simp[j] = rhss_deriv_simp[j].subs(var,0)
    zero_elements_to_remove = []
    # Create array of indices for expressions that are zero
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == sp.sympify(0):
            zero_elements_to_remove.append(i)

    # When removing terms that are zero, we need to take into account their new index (after each removal)
    count = 0
    for i in range(len(zero_elements_to_remove)):
        del lhss_deriv_simp[zero_elements_to_remove[i]+count]
        del rhss_deriv_simp[zero_elements_to_remove[i]+count]
        count -= 1
    return lhss_deriv_simp,rhss_deriv_simp

# Step 7: Call the simplication function and then copy results
lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv,rhss_deriv)
lhss_deriv = lhss_deriv_simp
rhss_deriv = rhss_deriv_simp

#TylerK: for debuggin
print("lhss_deriv = ",lhss_deriv)
print("rhss_deriv = ",rhss_deriv)

lhss_deriv =  [rprm, csiprm, s1dots2prm, uprm, omegaprm]
rhss_deriv =  [2*x*xprm + 2*y*yprm, pxprm + py*pzprm + pyprm*pz, s1x*s2xprm + s1xprm*s2x + s1y*s2yprm + s1yprm*s2y + s1z*s2zprm + s1zprm*s2z, -rprm/r**2, rprm/(2*sqrt(r))]


<a id='step8'></a>

# Step 8: Differentiate with respect to a specific free variable \[Back to [top](#toc)\]
$$\label{step8}$$

In [Step 6](#step6) we took a generic derivative of each term, assuming it is a function of the varible $\textrm{xx}$.  We now define a function that will select a specific free variable for differentiation.

In [8]:
def deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                 s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0):

    # Copy expressions into another array
    lhss_deriv_new = []
    rhss_deriv_new = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_new.append(lhss_deriv[i])
        rhss_deriv_new.append(rhss_deriv[i])
    # For each free symbol, replace it with the desired derivative
    for i in range(len(rhss_deriv_new)):
        for var in rhss_deriv_new[i].free_symbols:
            if str(var)=="xprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,xprm)
            elif str(var)=="yprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,yprm)
            elif str(var)=="zprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,zprm)
            elif str(var)=="pxprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,pxprm)
            elif str(var)=="pyprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,pyprm)
            elif str(var)=="pzprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,pzprm)
            elif str(var)=="s1xprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s1xprm)
            elif str(var)=="s1yprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s1yprm)
            elif str(var)=="s1zprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s1zprm)
            elif str(var)=="s2xprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s2xprm)
            elif str(var)=="s2yprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s2yprm)
            elif str(var)=="s2zprm":
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,s2zprm)
    # Simplify derivative expressions again
    lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv_new,rhss_deriv_new)
    return lhss_deriv_simp,rhss_deriv_simp

#def deriv_onevar_test(lhss_deriv,rhss_deriv,variable_list,variable):
def deriv_onevar_test(lhss_deriv,rhss_deriv,variable_list,index):
    variableprm_list = []
    for variable in variable_list:
        variableprm_list.append(str(variable)+"prm")

    # Copy expressions into another array
    lhss_deriv_new = []
    rhss_deriv_new = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_new.append(lhss_deriv[i])
        rhss_deriv_new.append(rhss_deriv[i])
    # For each free symbol, replace it with the desired derivative
    for i in range(len(rhss_deriv_new)):
        #for var in rhss_deriv_new[i].free_symbols:
        #for var in variable_list:
        for var in variableprm_list:
            if variableprm_list.index(str(var))==index:
            #if var==(variable+"prm"):
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,1)
            else:
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,0)
    # Simplify derivative expressions again
    lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv_new,rhss_deriv_new)
    return lhss_deriv_simp,rhss_deriv_simp

<a id='step9'></a>

# Step 9: Compute derivatives with respect to each free variable \[Back to [top](#toc)\]
$$\label{step9}$$

This needs to be made into a loop!

In [9]:
print("New routine")
#lhss_derivative = []
#rhss_derivative = []
lhss_derivative = {}
rhss_derivative = {}
for index in range(len(dynamic_variables)):
#for squirtle in dynamic_variables:
    lhss_temp,rhss_temp = deriv_onevar_test(lhss_deriv,rhss_deriv,dynamic_variables,index)
    #lhss_derivative,rhss_derivative = deriv_onevar_test(lhss_deriv,rhss_deriv,dynamic_variables,index)
    #lhss_derivative[dynamic_variables[index]],rhss_derivative[dynamic_variables[index]] = deriv_onevar_test(lhss_deriv,rhss_deriv,dynamic_variables,index)
    #lhss_deriv_partial,rhss_deriv_partial = deriv_onevar_test(lhss_deriv,rhss_deriv,dynamic_variables,index)
    #lhss_derivative.append(lhss_deriv_partial)
    #rhss_derivative.append(rhss_deriv_partial)
    #TylerK: for debuggin
    lhss_derivative[dynamic_variables[index]] = lhss_temp
    rhss_derivative[dynamic_variables[index]] = rhss_temp
    print("left-hand side is", lhss_temp)
    print("right-hand side is", rhss_temp)
print("left-hand side", lhss_derivative)
print("right-hand side", rhss_derivative)

print("Old routine")
lhss_deriv_x,rhss_deriv_x = deriv_onevar(lhss_deriv,rhss_deriv, xprm=1,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                         s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_y,rhss_deriv_y = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=1,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                         s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_z,rhss_deriv_z = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=1,pxprm=0,pyprm=0,pzprm=0,
                                         s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_px,rhss_deriv_px = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=1,pyprm=0,pzprm=0,
                                           s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_py,rhss_deriv_py = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=1,pzprm=0,
                                           s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_pz,rhss_deriv_pz = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=1,pzprm=1,
                                           s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_s1x,rhss_deriv_s1x = deriv_onevar(lhss_deriv,rhss_deriv, xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                             s1xprm=1,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_s1y,rhss_deriv_s1y = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                             s1xprm=0,s1yprm=1,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_s1z,rhss_deriv_s1z = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                             s1xprm=0,s1yprm=0,s1zprm=1,s2xprm=0,s2yprm=0,s2zprm=0)
lhss_deriv_s2x,rhss_deriv_s2x = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                             s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=1,s2yprm=0,s2zprm=0)
lhss_deriv_s2y,rhss_deriv_s2y = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                             s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=1,s2zprm=0)
lhss_deriv_s2z,rhss_deriv_s2z = deriv_onevar(lhss_deriv,rhss_deriv,xprm=0,yprm=0,zprm=0,pxprm=0,pyprm=0,pzprm=0,
                                             s1xprm=0,s1yprm=0,s1zprm=0,s2xprm=0,s2yprm=0,s2zprm=1)
#TylerK: for debuggin
print("left-hand side is", lhss_deriv_x)
print("right-hand side is", rhss_deriv_x)

New routine
left-hand side is [rprm, uprm, omegaprm]
right-hand side is [2*x, -rprm/r**2, rprm/(2*sqrt(r))]
left-hand side is [rprm, uprm, omegaprm]
right-hand side is [2*y, -rprm/r**2, rprm/(2*sqrt(r))]
left-hand side is []
right-hand side is []
left-hand side is [csiprm]
right-hand side is [1]
left-hand side is [csiprm]
right-hand side is [pz]
left-hand side is [csiprm]
right-hand side is [py]
left-hand side is [s1dots2prm]
right-hand side is [s2x]
left-hand side is [s1dots2prm]
right-hand side is [s2y]
left-hand side is [s1dots2prm]
right-hand side is [s2z]
left-hand side is [s1dots2prm]
right-hand side is [s1x]
left-hand side is [s1dots2prm]
right-hand side is [s1y]
left-hand side is [s1dots2prm]
right-hand side is [s1z]
left-hand side {'z': [], 's1x': [s1dots2prm], 's2y': [s1dots2prm], 'py': [csiprm], 'pz': [csiprm], 's2x': [s1dots2prm], 'px': [csiprm], 's1y': [s1dots2prm], 's1z': [s1dots2prm], 'x': [rprm, uprm, omegaprm], 'y': [rprm, uprm, omegaprm], 's2z': [s1dots2prm]}
right-ha

<a id='step10'></a>

# Step 10: Output result \[Back to [top](#toc)\]
$$\label{step10}$$

We write the resulting derivatives in C code.

In [10]:
for var in dynamic_variables:
    with open("dHreal_d"+str(var)+".txt", "w") as output:
        outstring = "/* SEOBNR Hamiltonian expression: */\n"
        outstringsp = ""
        outsplhs = []
        outsprhs = []
        for i in range(len(lr)):
            outstring += outputC(sp.sympify(lr[i].rhs),lr[i].lhs,"returnstring","outCverbose=False,includebraces=False,CSE_enable=False")
            outstringsp += lr[i].lhs+" = "+lr[i].rhs+"\n"
            outsplhs.append(sp.sympify(lr[i].lhs))
            outsprhs.append(sp.sympify(lr[i].rhs))
        outstring += "\n\n\n/* SEOBNR \partial_"+str(var)+" H expression: */\n"
        #for i in range(len(lhss_deriv_x)):
            #outstring += outputC(rhss_deriv_x[i],str(lhss_deriv_x[i]),"returnstring","outCverbose=False,includebraces=False,CSE_enable=False")
            #outstringsp += str(lhss_deriv_x[i])+" = "+str(rhss_deriv_x[i])+"\n"
            #outsplhs.append(lhss_deriv_x[i])
            #outsprhs.append(rhss_deriv_x[i])
        for i in range(len(lhss_derivative[var])):
            outstring += outputC(rhss_derivative[var][i],str(lhss_derivative[var][i]),"returnstring","outCverbose=False,includebraces=False,CSE_enable=False")
            outstringsp += str(lhss_derivative[var][i])+" = "+str(rhss_derivative[var][i])+"\n"
            outsplhs.append(lhss_derivative[var][i])
            outsprhs.append(rhss_derivative[var][i])
        output.write("%s" % outstring)

<a id='code_validation'></a>

# Step 11: Code Validation against `SEOBNR_Derivative_Routine` NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation}$$

<a id='latex_pdf_output'></a>

# Step 11: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-SEOBNR_Derivative_Routine.pdf](Tutorial-SEOBNR_Derivative_Routine.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [11]:
import os,sys                    # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("NRPyPN_shortcuts",location_of_template_file=os.path.join(".."))

Notebook output to PDF is only supported on Linux systems, with pdflatex installed.
